In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.interpolate import BSpline
import timeit

from commons.utils import get_knots
from commons.optimizer import single_cellGPS
from commons.objFunc import objf3D

In [ ]:
def get_RMS(A, ground_truths, times, knots, k):
    '''
    reconstructs spline using A, k, and knots then calculate RMS distance between the ground truth trajectory
    
    A: array of control points
    x, y, z: ground truth
    times: np.linspace(start_time, end_time, Number of time points)
    '''
    x, y, z = ground_truths.reshape((3, -1))
    A_x, A_y, A_z = A.reshape((3, -1))

    spline_x = BSpline(knots, A_x, k)
    spline_y = BSpline(knots, A_y, k)
    spline_z = BSpline(knots, A_z, k)
    predicted_x = spline_x(times)
    predicted_y = spline_y(times)
    predicted_z = spline_z(times)

    MSE = ((predicted_x - x) ** 2 + (predicted_y - y) ** 2 + (predicted_z - z) ** 2).mean()

    return np.sqrt(MSE)

# Calculate ground truth trajectory for helical trajectory

In [ ]:
eval_times = np.arange(0, 299951, 50)
x = 2 * np.sin((eval_times/1000) * 6 * np.pi / 180)
y = 2 * np.cos((eval_times/1000) * 6 * np.pi / 180)
z = (eval_times/1000)*0.02 - 3
ground_truths = np.vstack((x, y, z))

# Load data

In [ ]:
# Hyperparameter settings
k = 3  # degree of spline
lambda0 = 0.0025
d_bar = 20  # average number of LoRs in one spline interval
dmax1, dmax2 = 2.8, 0.6  # distance restrictions
options = {'maxiter': 2000, 'disp': False}


# Load data
LORs_dir = 'data/Helical_100Bq300s.csv'
LORs = np.genfromtxt(LORs_dir, delimiter=',')

N = int(np.round(LORs.shape[0] / d_bar)) + 3  # Number of basis functions
knots = get_knots(k , 0, 300000, N)
P1, P2, times = LORs.T[0:3, :], LORs.T[4:7, :], LORs.T[3, :]
a0 = np.zeros((3, N))

# Compare the previous and current algorithm

In [ ]:
# Backpropagation
A = a0.copy()
start_time = timeit.default_timer()
A = single_cellGPS(A, LORs, N, k, knots, lambda0, dmax1, options=options)
A = single_cellGPS(A, LORs, N, k, knots, lambda0, dmax2, options=options)
print(f"current algorithm reconstruction time: {timeit.default_timer() - start_time}")

# Numerical Gradient
B = a0.copy()
start_time = timeit.default_timer()
result = minimize(lambda s: objf3D(k, s, knots, P1, P2, times, dmax1, lambda0), B, method='BFGS', options=options)
B = result.x
result = minimize(lambda s: objf3D(k, s, knots, P1, P2, times, dmax2, lambda0), B, method='BFGS', options=options)
B = result.x
print(f"previous algorithm reconstruction time: {timeit.default_timer() - start_time}")

# Compare Accuracy
print(f"current algorithm RMS distance: {get_RMS(A, ground_truths, eval_times, knots, k)}")
print(f"previous algorithm RMS distance: {get_RMS(B, ground_truths, eval_times, knots, k)}")